In [1]:
import requests
from lxml import etree
import re

In [2]:
url = "https://is.cuni.cz/studium/rozvrhng/sez_predmet.php?skr=2017&sem=1&fak=11320&ustav_mode=text&ustav=32-KFPP"
response = requests.get(url)

In [3]:
#tree = html.fromstring(response.content)
tree = etree.fromstring(response.content, etree.HTMLParser())

In [4]:
rows = tree.xpath('//table[@id="table1"]/tr[@class!="head1"]')

In [5]:
vykaz = {}
for tr in rows:
    cls = tr.xpath("./@class")[0]
    
    if cls == "head2":
        head = tr.xpath("./td/a/span/text()")[0]
        vykaz[head] = {"podily":{}, "studenti":None}
        #print(head)
        
    if cls in ["row1", "row2"]:
        cells = tr.xpath("./td/descendant-or-self::*/text()")
        cells = ["".join(td.xpath("./descendant-or-self::*/text()")) for td in tr.xpath("./td")]
        
        try:
            minuty = int(cells[4])
        except(ValueError):
            minuty = {
                "90 Liché týdny ": 45,
                "90 Liché týdny  (sudé kalendářní)":45,
                "180 Liché týdny  (sudé kalendářní)":90,
                "180 Sudé týdny  (liché kalendářní)":90,
            }[cells[4]]
        
        if re.search("\(", cells[1]): 
            podily = [int(i)*0.01*minuty for i in re.findall("([0-9]+)", cells[1])]
            jmena = re.findall("([^, ()0-9]+[^()0-9]+[^()0-9 ]+)", cells[1])
        else:
            podily = [minuty]
            jmena = [cells[1]]
        for podil, jmeno in zip(podily, jmena):
            vykaz[head]["podily"][jmeno] = podil + vykaz[head]["podily"].get(jmeno, 0)
        vykaz[head]["studenti"] = int(cells[8])

In [115]:
for predmet, data in vykaz.items():
    if data["studenti"] == 0: continue
    print(predmet, data["studenti"], "studenti")
    for ucitel, podil in data["podily"].items():
        print(" "*4, ucitel, podil)

NEVF106 Řádkovací mikroskopie - STM, AFM 3 studenti
     Ošťádal Ivan, doc. RNDr., CSc. 90
NEVF108 Moderní trendy ve fyzice povrchů 3 studenti
     Mašek Karel, doc. RNDr., Dr. 90
NEVF112 Metody zpracování fyzikálních měření 3 studenti
     Johánek Viktor, RNDr., Ph.D. 45.0
     Mašek Karel, doc. RNDr., Dr. 45.0
NEVF114 Fyzika tenkých vrstev I 2 studenti
     Sobotík Pavel, doc. RNDr., CSc. 90
NEVF115 Elektronika pro fyziky 4 studenti
     Tichý Milan, prof. RNDr., DrSc. 90
NEVF121 Horké plazma, problematika fúze 1 studenti
     Plašil Radek, doc. RNDr., Ph.D. 90.0
     Glosík Juraj, prof. RNDr., DrSc. 0.0
NEVF122 Fyzika plazmatu I 6 studenti
     Glosík Juraj, prof. RNDr., DrSc. 72.0
     Dohnal Petr, Mgr., Ph.D. 9.0
     Roučka Štěpán, RNDr., Ph.D. 9.0
NEVF126 Vakuová fyzika 2 studenti
     Gronych Tomáš, RNDr., CSc. 90
     Pavlů Jiří, doc. RNDr., Ph.D. 45
NEVF129 Fyzika povrchů 4 studenti
     Pavluch Jiří, doc. RNDr., CSc. 63.00000000000001
     Nehasil Václav, doc. RNDr., Dr. 72.